# 🚀 Model Context Protocol (MCP) Servers Demo

Welcome to the **MCP Servers Demo** for Databricks AI Engineers! 🎯

## 📋 What is MCP?

The **Model Context Protocol (MCP)** is an open standard that enables AI applications to securely connect to external data sources and tools. Think of it as a bridge between your AI models and the world of data they need to access.

## 🎯 What You'll Learn

In this demo, we'll demonstrate:
- ⚙️ How to set up and connect to MCP servers
- 🔧 Working with built-in Databricks MCP tools (like Python execution)
- 🔍 Exploring custom MCP servers and tools
- 📊 Practical examples of tool invocation and response handling

Let's dive in! 🏊‍♂️

---

---

## 🔄 Setup: Enable Auto-reload

First, let's enable auto-reload to ensure any changes to imported modules are automatically reflected.

In [0]:
%load_ext autoreload
%autoreload 2

---

## 📦 Installation: Required Dependencies

Now we'll install all the necessary packages for working with MCP servers:
- **`mcp`**: Core MCP protocol implementation
- **`databricks-sdk`**: Databricks SDK with OpenAI integration
- **`mlflow`**: For ML model lifecycle management
- **`databricks-agents`**: Databricks AI agents framework
- **`databricks-mcp`**: Databricks-specific MCP client

⏳ This might take a few minutes to complete...

In [0]:
pip install -U "mcp>=1.9" "databricks-sdk[openai]" "mlflow>=3.1.0" "databricks-agents>=1.0.0" "databricks-mcp"

---

## 🔄 Restart Python Environment

After installing new packages, we need to restart the Python environment to ensure all dependencies are properly loaded.


In [0]:
import json
dbutils.library.restartPython()

## 🌐 Configure Workspace Connection

Let's establish a connection to your Databricks workspace and construct the MCP server URL. This URL will be used to access the built-in AI tools provided by Databricks.

The `system.ai` schema contains powerful tools like the Python code interpreter! 🐍

In [0]:

from databricks.sdk import WorkspaceClient
workspace_client = WorkspaceClient()
workspace_hostname = workspace_client.config.host
mcp_server_url = f"{workspace_hostname}/api/2.0/mcp/functions/system/ai"
mcp_server_url

---

## 🛠️ Helper Functions: MCP Client Operations

Here we define utility functions to:
- **Connect** to MCP servers and discover available tools 🔍
- **Invoke** MCP tools with parameters and handle responses 📡

These functions will be reused throughout the demo to interact with different MCP servers.

In [0]:
from databricks_mcp import DatabricksMCPClient
import nest_asyncio
nest_asyncio.apply()

# This snippet below uses the Unity Catalog functions MCP server to expose built-in
# AI tools under `system.ai`, like the `system.ai.python_exec` code interpreter tool
def test_connect_to_server_get_tools(mcp_server_url):
     mcp_client = DatabricksMCPClient(server_url=mcp_server_url, workspace_client=workspace_client)
     tools = mcp_client.list_tools()

     print(
         f"Discovered tools {[t.name for t in tools]} "
         f"from MCP server {mcp_server_url}"
     )
     return mcp_client, tools

def mcp_invoke_tool(mcp_client, tool, params):
     import json
     result = mcp_client.call_tool(
         tool, params
     )
     print(f"Called MCP tool {tool}")
     print(f"Response:\n ")
     [print(json.dumps(item, indent=2, ensure_ascii=False)) for item in json.loads(json.loads(result.content[0].text)['rows'][0][0])]
     

---

## 🔍 Discover Built-in AI Tools

Let's connect to the Databricks system AI MCP server and see what tools are available! 

Expected tools:
- **`system__ai__python_exec`**: A powerful Python code interpreter that can execute code in a sandboxed environment 🐍✨


---

## 📊 Inspect Tool Schema

Let's examine the detailed schema of the Python execution tool. This will show us:
- 📝 **Description**: What the tool does
- 🔧 **Input Schema**: What parameters it expects
- 📤 **Output Schema**: What it returns
- ⚙️ **Annotations**: Additional metadata


In [0]:
mcp_client, tools = test_connect_to_server_get_tools(mcp_server_url)

In [0]:
print(json.dumps(tools[0].__dict__, indent=2, default=str))


---

## 🚀 Test Python Code Execution

Time for our first tool invocation! Let's test the Python executor with a simple "Hello, world!" example.

**What happens here:**
1. We send Python code to the MCP server
2. The server executes it in a secure sandbox
3. We receive the output back as structured data

🔒 **Security Note**: The execution happens in a stateless, sandboxed environment for safety!

In [0]:
mcp_invoke_tool(mcp_client,tools[0].name, {"code": "print('Hello, world!')"})

---

## 🔄 Connect to Custom MCP Server

Now let's explore a **custom MCP server**! This one provides insurance-related tools:

**Server**: `mc/fidel` - Fidelidade Insurance Company tools  
**Expected Tool**: `mc__fidel__search_conditions` - Search insurance policy conditions 📋🔍

This demonstrates how MCP can be used to expose domain-specific business tools!


In [0]:
mcp_server_url = f"{workspace_hostname}/api/2.0/mcp/functions/mc/fidel"
mcp_client, tools = test_connect_to_server_get_tools(mcp_server_url)


---

## 📋 Examine Insurance Tool Schema

Let's inspect the insurance search tool to understand:
- 🎯 **Purpose**: What this business tool accomplishes
- 📥 **Input**: How to query insurance conditions
- 📊 **Output**: What kind of insurance data it returns


In [0]:
print(json.dumps(tools[0].__dict__, indent=2, default=str))


---

## 🔍 Query Insurance Conditions

Let's test the insurance search tool by querying for **hazard types** covered by insurance policies.

**Query**: "names of hazard types covered by my insurance"

This demonstrates how MCP can expose specialized business knowledge and make it accessible to AI systems! 🏢💼

In [0]:
mcp_invoke_tool(mcp_client,tools[0].name, {"my_query": "names of hazard types covered by my insurance"})

---

## 🔐 Authentication Setup

For connecting to **external MCP servers** (like Databricks Apps), we need authentication. 

**🛠️ Setup Instructions:**
1. Run this in your **local terminal**:
   ```bash
   databricks auth login --host https://adb-984752964297111.11.azuredatabricks.net
   Databricks profile name: mehdi-dbx-profile
   databricks auth token -p mehdi-dbx-profile
   ```
2. Copy the token and enter it when prompted below 🔑


In [0]:
import getpass
token = getpass.getpass("Enter your Databricks token: ")

---

## 🌐 Connect to External MCP Server App

Now we'll connect to a **Databricks App** that hosts a custom MCP server! This demonstrates:

- 🔗 **External Connectivity**: How to connect to MCP servers outside Unity Catalog
- 🛡️ **OAuth Authentication**: Secure connection using Databricks OAuth
- 📱 **App Integration**: How Databricks Apps can expose MCP tools

**Target App**: `custom-mcp-server-mehdi` - A simple calculator service

- Deploy an external MCP Server in Databricks Apps Following the steps provided in the following Repo :

https://github.com/databrickslabs/mcp/tree/master/examples/custom-server

In [0]:
import json
from databricks.sdk import WorkspaceClient
from databricks_mcp import DatabricksOAuthClientProvider
from databricks.sdk.credentials_provider import ModelServingUserCredentials
from mcp.client.streamable_http import streamablehttp_client as connect
from mcp import ClientSession

import logging
logging.getLogger('mcp.client.streamable_http').setLevel(logging.ERROR)

workspace_hostname = WorkspaceClient().config.host
MCP_SERVER_DATABRICKS_APP_DOMAIN = 'custom-mcp-server-mehdi-984752964297111.11.azure.databricksapps.com'

async def connect_to_custom_mcp_server_app_and_get_tools():
    
    app_url = f"https://{MCP_SERVER_DATABRICKS_APP_DOMAIN}/mcp"

    client = WorkspaceClient(
        host=workspace_hostname,
        token= token  # Replace with your actual token
    )

    async with connect(app_url, auth=DatabricksOAuthClientProvider(client)) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tools = await session.list_tools()
            print(f"Available tools: {[tool.name for tool in tools.tools]}")
            return session, tools



---

## 🧮 Test External MCP Server Tools

Let's explore and test the tools from our external Databricks App MCP server!

**Expected Tool**: `add` - A simple addition calculator

This demonstrates the full MCP workflow with external servers. 🔄

In [0]:
tools = await with_mcp_session(explore_tools)
result = await with_mcp_session(call_add_tool)

---

## 📊 Detailed Tool Schema (JSON)

Let's examine the complete JSON schema of our external tools for deeper understanding:


In [0]:
print(json.dumps([tool.__dict__ for tool in tools.tools], indent=2))

---

## 🐍 Pretty Print Tool Schema

For better readability, let's use Python's `pprint` to display the tool schema:




In [0]:
from pprint import pprint
pprint([tool.__dict__ for tool in tools.tools])

---

## 🌳 Beautiful Tool Visualization

Now let's create a beautiful tree visualization of our MCP tools using Rich! This provides:

- 🎯 **Clear structure**: Tool names, descriptions, and parameters
- 🎨 **Color coding**: Different colors for different types of information
- 📋 **Hierarchical view**: Easy to understand tool capabilities at a glance


In [0]:
%pip install rich -q


In [0]:
from rich import print
from rich.tree import Tree

tree = Tree("Tools")
for tool in tools.tools:
   tool_tree = tree.add(f"[bold blue]{tool.name}[/bold blue]: {tool.description}")
   
   # Input parameters
   input_tree = tool_tree.add("[green]Input Parameters[/green]")
   for param, details in tool.inputSchema.get('properties', {}).items():
       input_tree.add(f"{param}: {details.get('type', 'unknown')} - {details.get('title', '')}")
   
   # Output parameters  
   output_tree = tool_tree.add("[yellow]Output Parameters[/yellow]")
   for param, details in tool.outputSchema.get('properties', {}).items():
       output_tree.add(f"{param}: {details.get('type', 'unknown')} - {details.get('title', '')}")

print(tree)

---

## 📋 Final Result Analysis

Let's examine the detailed response from our MCP tool call to understand the complete response structure:

In [0]:
print(json.dumps(result.__dict__, indent=2, default=str))


---

# 🎉 Demo Summary

## 🏆 What We Accomplished

Congratulations! You've successfully explored the Model Context Protocol (MCP) ecosystem in Databricks! Here's what we covered:

### 🔧 **Built-in Tools Explored**
- ✅ **System AI Python Executor**: Executed Python code in a secure sandbox
- ✅ **Tool Discovery**: Connected to Unity Catalog MCP servers
- ✅ **Schema Inspection**: Examined tool input/output schemas

### 🏢 **Custom Business Tools**
- ✅ **Insurance Search Tool**: Queried domain-specific business data
- ✅ **Structured Responses**: Received formatted business intelligence
- ✅ **Real-world Integration**: Demonstrated practical business use cases

### 🌐 **External App Integration**
- ✅ **Databricks Apps**: Connected to external MCP servers
- ✅ **OAuth Authentication**: Secured connections with proper authentication
- ✅ **Calculator Service**: Tested external tool functionality

### 🎨 **Advanced Visualization**
- ✅ **Rich Formatting**: Created beautiful tool visualizations
- ✅ **Tree Structures**: Organized tool information hierarchically
- ✅ **Response Analysis**: Examined detailed tool responses

## 🚀 **Key Takeaways**

1. **🔌 Seamless Integration**: MCP provides a standardized way to connect AI systems to external tools and data sources

2. **🔒 Security First**: All tool executions happen in secure, sandboxed environments

3. **📊 Structured Data**: Tools return well-formatted, structured responses that are easy to process

4. **🏢 Business Ready**: MCP can expose domain-specific business tools and knowledge bases

5. **🌐 Flexible Architecture**: Supports both internal Unity Catalog tools and external Databricks Apps

## 🔮 **Next Steps**

- **Build Custom Tools**: Create your own MCP servers for specific business needs
- **Integrate with AI Agents**: Use these tools in AI agent workflows
- **Scale Deployments**: Deploy MCP servers across your organization
- **Explore Advanced Features**: Dive deeper into MCP capabilities and integrations

## 📚 **Additional Resources**

- [MCP Official Documentation](https://modelcontextprotocol.io/)
- [Databricks MCP Integration Guide](https://docs.databricks.com/aws/en/generative-ai/mcp/)
- [Unity Catalog Functions](https://docs.databricks.com/en/sql/language-manual/sql-ref-functions-builtin.html)

---

**Happy building with MCP! 🎯🚀**